<a href="https://colab.research.google.com/github/Computer-Vision-IIITH-2021/assignment-4-Avinash2468/blob/main/CV_ass4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libaries and defining all the functions we will use in our code 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from os import listdir
from os.path import isfile, join
from sklearn.feature_selection import SelectPercentile, f_classif
import random
from google.colab import drive
drive.mount('/content/drive')

###############

def integral_image(img): #To calculate the integral image.
    
    height, width = img.shape[0], img.shape[1]
    integ_img = np.zeros_like(img).astype(np.uint32)
    
    for i in range(width):
        for j in range(height):
            
            if i-1<0 and j-1<0:
                integ_img[i,j] = img[i,j]
                
            elif i-1<0:
                integ_img[i,j] = integ_img[i,j-1] + img[i,j]
            
            elif j-1<0:
                integ_img[i,j] = integ_img[i-1,j] + img[i,j]
            
            else:
                integ_img[i,j] = integ_img[i-1,j] + integ_img[i,j-1] - integ_img[i-1,j-1] + img[i,j]
    
    return integ_img

###############

class sum_rectangle_region: # Finding the value of a feature
    def __init__(self,i,j,w,h):
        self.i = i
        self.j = j
        self.w = w
        self.h = h
        
    def compute_area(self,integ_img):
        
        top_left = integ_img[self.i,self.j]
        top_right = integ_img[self.i,self.j+self.w]
        bottom_left = integ_img[self.i+self.h,self.j]
        bottom_right = integ_img[self.i+self.h,self.j+self.w]
        final_area = bottom_right + top_left -(top_right+bottom_left)
        
        return final_area

###############

def features(size): # Getting the features
    features_list = []
    
    for i in range(size):
        for j in range(size):       
            h = 1
            while i+h<size:
                w = 1
                while j+w<size:
                    
                    ########### Edge Features
                    
                    if i+2*h<size: # Vertically stacked rectangles
                        neg_reg = sum_rectangle_region(i,j,w,h)
                        pos_reg = sum_rectangle_region(i+h,j,w,h)
                        features_list.append([[pos_reg],[neg_reg]])
                        
                    if j+2*w<size: # Horizontally stacked rectangles
                        neg_reg = sum_rectangle_region(i,j,w,h)
                        pos_reg = sum_rectangle_region(i,j+w,w,h)
                        features_list.append([[pos_reg],[neg_reg]])
                        
                    ########### Line Features
                    
                    if i+3*h< size: # Vertically stacked rectangles
                        neg_reg1 = sum_rectangle_region(i,j,w,h)
                        neg_reg2 = sum_rectangle_region(i+2*h,j,w,h)
                        pos_reg = sum_rectangle_region(i+h,j,w,h)
                        features_list.append([[pos_reg],[neg_reg1,neg_reg2]])
                        
                    if j+3*w<size: # Horizontally stacked rectangles
                        neg_reg1 = sum_rectangle_region(i,j,w,h)
                        neg_reg2 = sum_rectangle_region(i,j+2*w,w,h)
                        pos_reg = sum_rectangle_region(i,j+w,w,h)
                        features_list.append([[pos_reg],[neg_reg1,neg_reg2]])
                        
                    ########### 4-rectangle Features
                    
                    if i+2*h<size and j+2*w<size:
                        neg_reg1 = sum_rectangle_region(i,j,w,h)
                        neg_reg2 = sum_rectangle_region(i+h,j+w,w,h)
                        pos_reg1 = sum_rectangle_region(i,j+w,w,h)
                        pos_reg2 = sum_rectangle_region(i+h,j,w,h)
                        features_list.append([[pos_reg1,pos_reg2],[neg_reg1,neg_reg2]])
                        
                    w+=1
                h+=1
    return features_list

###############

def init_feature_image_matrix(features_list,integral_image_list, no_faces, no_nofaces): #Initialising various matrices

  no_feats = len(features_list)
  no_imgs = len(integral_image_list)
  print("The number of features are %d and the number of images are %d"%(no_feats,no_imgs))

  feature_image = np.zeros((no_feats,no_imgs))
  class_feature_image = np.zeros((1,no_imgs))
  weights = np.zeros((1,no_imgs))
  print(feature_image.shape)

  for i in range(no_feats):
    for j in range(no_imgs):
      
      feature_sum = 0
      
      for posregs in features_list[i][0]:
          feature_sum+=posregs.compute_area(integral_image_list[j][0])
          
      for negregs in features_list[i][1]:
          feature_sum-=negregs.compute_area(integral_image_list[j][0])
      
      class_feature_image[0,j] = integral_image_list[j][1]

      if integral_image_list[j][1] == 0:
        weights[0,j] = 1./(2*no_faces)
      else:
        weights[0,j] = 1./(2*no_nofaces)
      feature_image[i,j] = feature_sum

  return feature_image, weights, class_feature_image

###############

def select_best_classifier(): # Selecting the best classifier/feature
  tpos_wgts = 0
  tneg_wgts = 0

  for i in range(classes_image.shape[1]): # Getting the total weights of each class
    if classes_image[0,i] == 0:
      tpos_wgts+= weights_image[0,i]
    else:
      tneg_wgts+= weights_image[0,i]
  
  best_feature_index = -1 # To store the best weak classifier index
  feature_error = np.inf # The best weak classifier's error
  best_polarity = 0
  best_threshold = 0
  error_array = np.zeros((1,feature_image_matrix.shape[1]))
  start_time = time.time()

  for i in range(feature_image_matrix.shape[0]): # Going through each feature

    images_for_specific_feature = feature_image_matrix[i,:] # Feature value per image

    total_set = zip(images_for_specific_feature, weights_image[0,:], classes_image[0,:])
    sorted_total_set = sorted(total_set, key=lambda x: x[0])

    error_final, positive_below, negative_below, psofar, nsofar, thresh, polarity= np.inf, 0, 0, 0, 0, None, None # Initialising the values
    
    for fi, w, c in sorted_total_set:
      
      error = min(positive_below + tneg_wgts - negative_below, negative_below + tpos_wgts - positive_below)
      # error = min(negative_below + tpos_wgts - positive_below, positive_below + tneg_wgts - negative_below)
      if error<error_final:
        error_final = error

        thresh = fi
        if psofar>nsofar:
          polarity = 1
        else:
          polarity = -1

      if int(c) == 0:
        positive_below+=w
        psofar+=1
      else:
        negative_below+=w
        nsofar+=1

    #  feat_error = np.sum(weights_image[0,:][polarity*images_for_specific_feature<polarity*thresh]) 

    feat_error = 0
    errorarray_temp = np.zeros((1,feature_image_matrix.shape[1]))
    for index, ii in enumerate(images_for_specific_feature):

      if polarity*ii<polarity*thresh and classes_image[0,:][index] == 1:
        feat_error+=weights_image[0,:][index]
        errorarray_temp[0,index] = 1
      elif polarity*ii>polarity*thresh and classes_image[0,:][index] == 0:
        feat_error+=weights_image[0,:][index]
        errorarray_temp[0,index] = 1

    if feat_error<feature_error:
      feature_error = feat_error
      best_feature_index = i
      error_array = errorarray_temp
      best_polarity = polarity
      best_threshold = thresh

  print("--- %s seconds ---" % (time.time() - start_time))

  return feature_error, best_feature_index, error_array, best_polarity, best_threshold

########

def test_algo(integral_image_list,final_best_classifiers,polarity_list,threshold_list,beta_list): #Code to test the accuracy
  count = 0

  for index1, j in enumerate(integral_image_list):
    final_value = 0
    alpha_sum = 0
    for index,i in enumerate(final_best_classifiers):

      feature_sum = 0
    
      for posregs in features_list[i][0]:
          feature_sum+=posregs.compute_area(j[0])
          
      for negregs in features_list[i][1]:
          feature_sum-=negregs.compute_area(j[0])

      polarity = polarity_list[index]
      threshold = threshold_list[index]
      alpha_sum+= np.log(1/beta_list[index])

      if polarity*feature_sum<polarity*threshold:
          final_value+= np.log(1/beta_list[index])
  

    if final_value >= 0.5*alpha_sum and classes_image[0,index1] == 0:
      count+=1
    elif final_value < 0.5*alpha_sum and classes_image[0,index1] == 1:
      count+=1

  return count

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Code to 

1. Get the images.
2. Divide them into train/test.
3. Generate the integral images.
4. Initialise various matrices which we need in our code.

# Do not run unless you want to retrain the entire model (will take 2-3 hours)

In [2]:
### Do not run unless you want to train from scratch

size = 9

mypath_face = "/content/drive/MyDrive/faces/face.train/face/train/face/"
mypath_noface = "/content/drive/MyDrive/faces/face.train/face/train/non-face/"

onlyfiles_face = [f for f in listdir(mypath_face) if isfile(join(mypath_face, f))][:1000]
onlyfiles_noface = [f for f in listdir(mypath_noface) if isfile(join(mypath_noface, f))][:1000]

random.shuffle(onlyfiles_face)
random.shuffle(onlyfiles_noface)

train_onlyfiles_face = onlyfiles_face[:900]
train_onlyfiles_noface = onlyfiles_noface[:900]

test_onlyfiles_face = onlyfiles_face[900:]
test_onlyfiles_noface = onlyfiles_noface[900:]


no_faces = len(train_onlyfiles_face)
no_nofaces = len(train_onlyfiles_noface)
no_totalfaces = no_faces + no_nofaces

print("There are %d training images with faces"%(len(train_onlyfiles_face)))
print("There are %d training images with no faces"%(len(train_onlyfiles_noface)))
integral_image_list = []

count = 0
for f in train_onlyfiles_face:
  image_file = mypath_face + f
  img = cv2.imread(image_file,0)
  img = cv2.resize(img,(size,size))
  integral_image_list.append([integral_image(img),0])
  count+=1
  if count%100 == 0:
    print(count)

count = 0
for f in train_onlyfiles_noface:
  image_file = mypath_noface + f
  img = cv2.imread(image_file,0)
  img = cv2.resize(img,(size,size))
  integral_image_list.append([integral_image(img),1])
  count+=1
  if count%100 == 0:
    print(count)

features_list = features(size)
start_time = time.time()
feature_image_matrix, weights_image, classes_image = init_feature_image_matrix(features_list,integral_image_list, no_faces, no_nofaces)
print("--- %s seconds ---" % (time.time() - start_time))

There are 900 training images with faces
There are 900 training images with no faces
100
200
300
400
500
600
700
800
900
100
200
300
400
500
600
700
800
900
The number of features are 2056 and the number of images are 1800
(2056, 1800)
--- 31.600548267364502 seconds ---


# Code to save all the matrices so that we don't have to retrain everytime.

In [3]:
np.save( "/content/drive/MyDrive/faces/feature_image.npy",feature_image_matrix)
np.save( "/content/drive/MyDrive/faces/weights_image.npy",weights_image)
np.save( "/content/drive/MyDrive/faces/classes_image.npy",classes_image)
np.save( "/content/drive/MyDrive/faces/integral_image.npy",np.array(integral_image_list))
np.save( "/content/drive/MyDrive/faces/features.npy",np.array(features_list))
np.save( "/content/drive/MyDrive/faces/test_face.npy",np.array(test_onlyfiles_face))
np.save( "/content/drive/MyDrive/faces/test_noface.npy",np.array(test_onlyfiles_noface))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


# Code to open all the saved matrices so that we don't have to retrain everytime.

In [12]:
feature_image_matrix = np.load("/content/drive/MyDrive/faces/feature_image.npy")
weights_image = np.load( "/content/drive/MyDrive/faces/weights_image.npy")
classes_image = np.load("/content/drive/MyDrive/faces/classes_image.npy")
integral_image_list = np.load("/content/drive/MyDrive/faces/integral_image.npy",allow_pickle = True)
features_list = np.load("/content/drive/MyDrive/faces/features.npy",allow_pickle = True)
test_onlyfiles_face = np.load("/content/drive/MyDrive/faces/test_face.npy",allow_pickle = True)
test_onlyfiles_noface =  np.load("/content/drive/MyDrive/faces/test_noface.npy",allow_pickle = True)

# Code to get the best classifiers
# Do not run unless you want to retrain the entire model (will take 2-3 hours)

In [5]:
# indices = SelectPercentile(f_classif, percentile=10).fit(feature_image_matrix.T,classes_image[0,:]).get_support(indices=True)
# feature_image_matrix = feature_image_matrix[indices]
# print(feature_image_matrix.shape)

final_best_classifiers = []
beta_list = []
polarity_list = []
threshold_list =[]
T = 10
for i in range(T):
  print(i)
  # weights_image = weights_image / np.linalg.norm(weights_image)
  # feature_error, best_feature_index, error_array, best_polarity, best_threshold = select_best_classifier(feature_image_matrix, weights_image, classes_image)
  feature_error, best_feature_index, error_array, best_polarity, best_threshold = select_best_classifier()
  print(best_feature_index)
  final_best_classifiers.append(best_feature_index)
  polarity_list.append(best_polarity)
  threshold_list.append(best_threshold)
  beta = feature_error/(1-feature_error)
  beta_list.append(beta)
  for index,i in enumerate(weights_image[0,:]):
    weights_image[0,index] = weights_image[0,index]*(beta**(1-(error_array[0,index])))

0
--- 16.82512640953064 seconds ---
500
1
--- 16.38855218887329 seconds ---
694
2
--- 17.10617184638977 seconds ---
1132
3
--- 16.940529823303223 seconds ---
423
4
--- 17.123265743255615 seconds ---
445
5
--- 16.96337580680847 seconds ---
381
6
--- 17.12041664123535 seconds ---
354
7
--- 16.90467643737793 seconds ---
423
8
--- 17.15509819984436 seconds ---
445
9
--- 16.858633995056152 seconds ---
429


# Code to save all the matrices so that we don't have to retrain everytime.

In [6]:
np.save( "/content/drive/MyDrive/faces/final_best_classifiers.npy",final_best_classifiers)
np.save( "/content/drive/MyDrive/faces/polarity_list.npy",polarity_list)
np.save( "/content/drive/MyDrive/faces/threshold_list.npy",threshold_list)
np.save( "/content/drive/MyDrive/faces/beta_list.npy",np.array(beta_list))

# Code to open all the saved matrices so that we don't have to retrain everytime.

In [13]:
final_best_classifiers = np.load("/content/drive/MyDrive/faces/final_best_classifiers.npy")
polarity_list = np.load("/content/drive/MyDrive/faces/polarity_list.npy")
threshold_list =  np.load("/content/drive/MyDrive/faces/threshold_list.npy")
beta_list = np.load("/content/drive/MyDrive/faces/beta_list.npy")

# Training Accuracy

In [14]:
count = test_algo(integral_image_list,final_best_classifiers,polarity_list,threshold_list,beta_list)
print(no_totalfaces)
print("The accuracy of the model is: %f"%(count/1800))

200
The accuracy of the model is: 0.593333


# Testing Accuracy

In [16]:
### reading the image

# mypath_face = "/content/drive/MyDrive/faces/face.test/face/test/face/"
# mypath_noface = "/content/drive/MyDrive/faces/face.test/face/test/non-face/"

mypath_face = "/content/drive/MyDrive/faces/face.train/face/train/face/"
mypath_noface = "/content/drive/MyDrive/faces/face.train/face/train/non-face/"

# test_onlyfiles_face = []
# test_onlyfiles_noface = []
# test_onlyfiles_face = [f for f in listdir(mypath_face) if isfile(join(mypath_face, f))]
# test_onlyfiles_noface = [f for f in listdir(mypath_noface) if isfile(join(mypath_noface, f))][:200]

no_faces = len(test_onlyfiles_face)
no_nofaces = len(test_onlyfiles_noface)
no_totalfaces = no_faces + no_nofaces

classes_image = np.zeros((1,no_totalfaces))
classes_image[0,no_faces:] = 1

integral_image_list = []

count = 0
for f in test_onlyfiles_face:
  image_file = mypath_face + f
  img = cv2.imread(image_file,0)
  # img = cv2.resize(img,(size,size))
  integral_image_list.append([integral_image(img),0])

count = 0
for f in test_onlyfiles_noface:
  image_file = mypath_noface + f
  img = cv2.imread(image_file,0)
  # img = cv2.resize(img,(size,size))
  integral_image_list.append([integral_image(img),1])

count = test_algo(integral_image_list,final_best_classifiers,polarity_list,threshold_list,beta_list)
print(no_totalfaces)
print("The accuracy of the model is: %f"%(count/no_totalfaces))

200
The accuracy of the model is: 0.522222
